# Import the Zenbase Library

In [4]:
import sys
import subprocess

def install_package(package):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    except subprocess.CalledProcessError as e:
        print(f"Failed to install {package}: {e}")
        raise

def install_packages(packages):
    for package in packages:
        install_package(package)

try:
    # Check if running in Google Colab
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    # Install the zenbase package if running in Google Colab
    # install_package('zenbase')
    # Install the zenbse package from a GitHub branch if running in Google Colab
    install_package('git+https://github.com/zenbase-ai/lib.git@main#egg=zenbase&subdirectory=py')

    # List of other packages to install in Google Colab
    additional_packages = [
        'python-dotenv',
        'langsmith[vcr]',
        'openai',
        'langchain',
        'langchain_openai'
    ]
    
    # Install additional packages
    install_packages(additional_packages)

# Now import the zenbase library
try:
    import zenbase
except ImportError as e:
    print("Failed to import zenbase: ", e)
    raise

# Configure the Environment

In [5]:
from pathlib import Path
from dotenv import load_dotenv

# import os
#
# os.environ["OPENAI_API_KEY"] = "..."
# os.environ["LANGCHAIN_API_KEY"] = "..."
# os.environ["LANGCHAIN_TRACING_V2"] = "true"

load_dotenv(Path("../../.env.test"), override=True)

True

In [6]:
import nest_asyncio

nest_asyncio.apply()

# Initial Setup

In [7]:
from langsmith.wrappers import wrap_openai
from openai import OpenAI

openai = wrap_openai(OpenAI())

# Now, you probably already have some LLM code.

It could use the OpenAI SDK, LangChain, or anything really. But it looks something like this:

In [11]:
import json
from langsmith import traceable
from langsmith.schemas import Run, Example
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

@traceable
def solver(inputs):
    messages = [
        ("system", """You are an expert math solver. Solve the given problem using the provided plan and operations.
        Return only the final numerical answer, without any additional text or explanation."""),
        ("user", "Question: {question}"),
        ("user", "Plan: {plan}"),
        ("user", "Mathematical Operation: {operation}"),
        ("user", "Provide the final numerical answer:")
    ]

    chain = ChatPromptTemplate.from_messages(messages) | ChatOpenAI(model="gpt-3.5-turbo") | StrOutputParser()

    plan = planner_chain(inputs)
    operation = operation_finder({"plan": plan["plan"], "question": inputs["question"]})
    
    inputs_to_answer = {
        "question": inputs["question"],
        "plan": plan["plan"],
        "operation": operation["operation"],
    }
    answer = chain.invoke(inputs_to_answer)
    return {"answer": answer}

@traceable
def planner_chain(inputs):
    messages = [
        ("system", """You are an expert math solver. Create a step-by-step plan to solve the given problem.
        Be clear and concise in your steps."""),
        ("user", "Problem: {question}\n\nProvide a step-by-step plan to solve this problem:")
    ]

    chain = ChatPromptTemplate.from_messages(messages) | ChatOpenAI(model="gpt-3.5-turbo") | StrOutputParser()
    plan = chain.invoke(inputs)
    return {"plan": plan}

@traceable
def operation_finder(inputs):
    messages = [
        ("system", """You are an expert math solver. Identify the overall mathematical operation needed to solve the problem 
        based on the given plan. Use simple operations like addition, subtraction, multiplication, and division."""),
        ("user", "Question: {question}"),
        ("user", "Plan: {plan}"),
        ("user", "Identify the primary mathematical operation needed:")
    ]

    chain = ChatPromptTemplate.from_messages(messages) | ChatOpenAI(model="gpt-3.5-turbo") | StrOutputParser()
    operation = chain.invoke(inputs)
    return {"operation": operation}

## And let's say you have an eval function like this

In [12]:
def score_answer(run: Run, example: Example):
    output = run.outputs["answer"].split("#### ")[-1]
    target = example.outputs["answer"].split("#### ")[-1]
    return {
        "key": "correctness",
        "score": int(output == target),
    }

## Then you're probably evaluating like this

In [13]:
# Evaluate using LangSmith
from langsmith import Client, evaluate

langsmith = Client()
evalset = list(langsmith.list_examples(dataset_name="GSM8K_test_set_langsmith_dataset_2j24kEFx8T718mqwRblcoNK3S0L"))

evaluate_kwargs = dict(
    data=evalset,
    evaluators=[score_answer],
    client=langsmith,
    max_concurrency=2,
)

evaluate(solver, **evaluate_kwargs)

View the evaluation results for experiment: 'proper-pain-78' at:
https://smith.langchain.com/o/05b1b207-85a0-5cf8-81b0-ed015fce6e1e/datasets/2e9517c2-10a2-421d-be30-d55f9d83ce1e/compare?selectedSessions=5cc32459-96bf-4508-a1e3-7c6314e6b218


0it [00:00, ?it/s]

<ExperimentResults proper-pain-78>

 # Now, how can we optimize this score?

## First, initialize the Zenbase ZenbaseTracer

In [20]:
from zenbase.core.managers import ZenbaseTracer
zenbase_tracer = ZenbaseTracer()


## Hook up Zenbase to your functions

1. Use the `zenbase_tracer` decorator.
2. Change function inputs to request
3. Use request's `zenbase.task_demos` to get the few-shot examples for the task and add them however you would like into your prompt.
4. If you need to use just a few examples, you can use `request.zenbase.task_demos[:2]` to get the first two examples.

In [21]:
from zenbase.types import LMRequest


@zenbase_tracer  # it is 1
@traceable
def solver(request: LMRequest):  # it is 2
    messages = [
        ("system", """You are an expert math solver. Solve the given problem using the provided plan and operations.
        Return only the final numerical answer, without any additional text or explanation."""),
    ]

    for demo in request.zenbase.task_demos: # it is 3
        messages += [
            ("user", f'Example Question: {demo.inputs["question"]}'),
            ("assistant", f'Example Answer: {demo.outputs["answer"]}'),
        ] # it is 4
    
    messages.extend([
        ("user", "Question: {question}"),
        ("user", "Plan: {plan}"),
        ("user", "Mathematical Operation: {operation}"),
        ("user", "Provide the final numerical answer:")
    ])

    chain = ChatPromptTemplate.from_messages(messages) | ChatOpenAI(model="gpt-3.5-turbo") | StrOutputParser()

    plan = planner_chain(request.inputs)
    operation = operation_finder({"plan": plan["plan"], "question": request.inputs["question"]})

    inputs_to_answer = {
        "question": request.inputs["question"],
        "plan": plan["plan"],
        "operation": operation["operation"],
    }
    answer = chain.invoke(inputs_to_answer)
    return {"answer": answer}


@zenbase_tracer  # it is 1
@traceable
def planner_chain(request: LMRequest):  # it is 2
    messages = [
        ("system", """You are an expert math solver. Create a step-by-step plan to solve the given problem.
        Be clear and concise in your steps."""),
        ("user", "Problem: {question}\n\nProvide a step-by-step plan to solve this problem:")
    ]

    if request.zenbase.task_demos:  # it is 3
        for demo in request.zenbase.task_demos[:2]:  # it is 4
            messages += [
                ("user", demo.inputs["question"]),
                ("assistant", demo.outputs["plan"]),
            ]

    chain = ChatPromptTemplate.from_messages(messages) | ChatOpenAI(model="gpt-3.5-turbo") | StrOutputParser()
    plan = chain.invoke(request.inputs)
    return {"plan": plan}


@zenbase_tracer  # it is 1
@traceable
def operation_finder(request: LMRequest):  # it is 2
    messages = [
        ("system", """You are an expert math solver. Identify the overall mathematical operation needed to solve the problem 
        based on the given plan. Use simple operations like addition, subtraction, multiplication, and division."""),
        ("user", "Question: {question}"),
        ("user", "Plan: {plan}"),
        ("user", "Identify the primary mathematical operation needed:")
    ]

    if request.zenbase.task_demos:  # it is 3 
        for demo in request.zenbase.task_demos[:2]:  # it is 4
            messages += [
                ("user", demo.inputs["question"]),
                ("user", demo.inputs["plan"]),
                ("assistant", demo.outputs["operation"]),
            ]

    chain = ChatPromptTemplate.from_messages(messages) | ChatOpenAI(model="gpt-3.5-turbo") | StrOutputParser()
    operation = chain.invoke(request.inputs)
    return {"operation": operation}

## Now we can optimize!

### Set up your optimizer:

In [22]:
from zenbase.adaptors.langchain import ZenLangSmith
from zenbase.optim.metric.bootstrap_few_shot import BootstrapFewShot

# Define your Langsmith and helper
langsmith = Client()
zen_langsmith_adaptor = ZenLangSmith(client=langsmith)

TRAIN_SET = "GSM8K_train_set_langsmith_dataset_2j24jtX6pr5OFyi9IRlj2Pk29NX"
TEST_SET = "GSM8K_test_set_langsmith_dataset_2j24kEFx8T718mqwRblcoNK3S0L"
VALIDATION_SET = "GSM8K_validation_set_langsmith_dataset_2j24kCxc6WhYDe27hEcnhUEHPmE"
SHOTS = 2
SAMPLES = 2

train_set = zen_langsmith_adaptor.fetch_dataset(dataset_name=TRAIN_SET)
test_set = zen_langsmith_adaptor.fetch_dataset(dataset_name=TEST_SET)
validation_set = zen_langsmith_adaptor.fetch_dataset(dataset_name=VALIDATION_SET)

evaluator_kwargs = dict(
    evaluators=[score_answer],
    client=langsmith,
    max_concurrency=1,
)

bootstrap_few_shot = BootstrapFewShot(
    shots=SHOTS,
    training_set=train_set,
    test_set=test_set,
    validation_set=validation_set,
    evaluator_kwargs=evaluator_kwargs,
    zen_adaptor=zen_langsmith_adaptor,
)




### Do the optimization

In [23]:
best_fn, candidates = bootstrap_few_shot.perform(
    solver,
    samples=SAMPLES,
    rounds=1,
    trace_manager=zenbase_tracer,
)

View the evaluation results for experiment: 'zenbase-cross-group-eco-centric-696109d5' at:
https://smith.langchain.com/o/05b1b207-85a0-5cf8-81b0-ed015fce6e1e/datasets/2e9517c2-10a2-421d-be30-d55f9d83ce1e/compare?selectedSessions=fe2556bb-b692-49e8-977e-c5d7caf62e02


0it [00:00, ?it/s]

View the evaluation results for experiment: 'zenbase-progressive-grid-enabled-41553c50' at:
https://smith.langchain.com/o/05b1b207-85a0-5cf8-81b0-ed015fce6e1e/datasets/f4949b35-e36c-49b8-9e07-35d4779eb5f7/compare?selectedSessions=40892a37-83d4-48fd-822c-8e39b6e5ace2


0it [00:00, ?it/s]

View the evaluation results for experiment: 'zenbase-extended-fresh-thinking-7910468b' at:
https://smith.langchain.com/o/05b1b207-85a0-5cf8-81b0-ed015fce6e1e/datasets/f4949b35-e36c-49b8-9e07-35d4779eb5f7/compare?selectedSessions=80fea70e-7e7b-4a40-ab82-509773a7b884


0it [00:00, ?it/s]

View the evaluation results for experiment: 'zenbase-ameliorated-analyzing-b4eba06e' at:
https://smith.langchain.com/o/05b1b207-85a0-5cf8-81b0-ed015fce6e1e/datasets/c608cbe2-12f7-4953-8a6b-8810884c6042/compare?selectedSessions=ddd0e089-7c47-4c46-baf7-b72f4e91a87a


0it [00:00, ?it/s]

View the evaluation results for experiment: 'zenbase-switchable-6thgeneration-9cc158ce' at:
https://smith.langchain.com/o/05b1b207-85a0-5cf8-81b0-ed015fce6e1e/datasets/2e9517c2-10a2-421d-be30-d55f9d83ce1e/compare?selectedSessions=f70854f6-9490-42b7-b18f-b8ee46c089da


0it [00:00, ?it/s]

### Introspect evaluation improvement

You can see in this example that the best function has improved the evaluation score by 50%.

In [29]:
bootstrap_few_shot.base_evaluation.evals

{'score': 0.5}


In [31]:
bootstrap_few_shot.best_evaluation.evals

{'score': 1.0}


### Use your optimized function

In [ ]:
zenbase_tracer.all_traces = {}

In [44]:
# Now you can use your zenbase fn
best_fn({"question": "If I have 30% of shares, and Mo has 24.5% of shares, how many of our 10M shares are unassigned?"})

4,550,000


### Introspect function traces

In [38]:
function_traces = [v for k, v in zenbase_tracer.all_traces.items()][0]["optimized"]

### Check the optimized parameters for solver


In [39]:
from pprint import pprint

pprint(function_traces["solver"]["args"]["request"].zenbase.task_demos)


[LMDemo(inputs={'question': 'Natalia sold clips to 48 of her friends in April, '
                            'and then she sold half as many clips in May. How '
                            'many clips did Natalia sell altogether in April '
                            'and May?'},
        outputs={'answer': '72'},
        adaptor_object=None),
 LMDemo(inputs={'question': 'Betty is saving money for a new wallet which '
                            'costs $100. Betty has only half of the money she '
                            'needs. Her parents decided to give her $15 for '
                            'that purpose, and her grandparents twice as much '
                            'as her parents. How much more money does Betty '
                            'need to buy the wallet?'},
        outputs={'answer': '5'},
        adaptor_object=None),
 LMDemo(inputs={'question': 'Julie is reading a 120-page book. Yesterday, she '
                            'was able to read 12 pages and today

### Check the optimized parameters for operation_finder


In [37]:
from pprint import pprint

pprint(function_traces["operation_finder"]["args"]["request"].zenbase.task_demos)

[LMDemo(inputs={'plan': 'Step 1: Calculate how many clips Natalia sold in May '
                        'by finding half of the clips she sold in April.\n'
                        'Number of clips sold in May = 48 / 2\n'
                        '\n'
                        'Step 2: Add the number of clips sold in April and May '
                        'to find the total number of clips sold.\n'
                        'Total number of clips sold = Number of clips sold in '
                        'April + Number of clips sold in May\n'
                        '\n'
                        'Step 3: Substitute the values found in step 1 into '
                        'step 2 and calculate the total number of clips sold '
                        'by Natalia in April and May.',
                'question': 'Natalia sold clips to 48 of her friends in April, '
                            'and then she sold half as many clips in May. How '
                            'many clips did Natalia se

### Check the optimized parameters for planner_chain


In [40]:
from pprint import pprint

pprint(function_traces["planner_chain"]["args"]["request"].zenbase.task_demos)

[LMDemo(inputs={'question': 'Natalia sold clips to 48 of her friends in April, '
                            'and then she sold half as many clips in May. How '
                            'many clips did Natalia sell altogether in April '
                            'and May?'},
        outputs={'plan': 'Step 1: Calculate how many clips Natalia sold in May '
                         'by finding half of the clips she sold in April.\n'
                         'Number of clips sold in May = 48 / 2\n'
                         '\n'
                         'Step 2: Add the number of clips sold in April and '
                         'May to find the total number of clips sold.\n'
                         'Total number of clips sold = Number of clips sold in '
                         'April + Number of clips sold in May\n'
                         '\n'
                         'Step 3: Substitute the values found in step 1 into '
                         'step 2 and calculate the total nu

## How to save the function and load it later

### Save the optimized function args to a file

In [41]:
bootstrap_few_shot.save_optimizer_args("bootstrap_few_shot_args.zenbase")

### Load the optimized function args with the function

In [42]:
optimized_function = bootstrap_few_shot.load_optimizer_and_function("bootstrap_few_shot_args.zenbase", solver, zenbase_tracer)

### Use the loaded function and make sure it loaded the demos.

In [43]:
zenbase_tracer.all_traces = {}
optimized_function({"question": "If I have 30% of shares, and Mo has 24.5% of shares, how many of our 10M shares are unassigned?"})
function_traces = [v for k, v in zenbase_tracer.all_traces.items()][0]["optimized"]
from pprint import pprint

pprint(function_traces["solver"]["args"]["request"].zenbase.task_demos)
pprint(function_traces["planner_chain"]["args"]["request"].zenbase.task_demos)
pprint(function_traces["operation_finder"]["args"]["request"].zenbase.task_demos)


[LMDemo(inputs={'question': 'Natalia sold clips to 48 of her friends in April, '
                            'and then she sold half as many clips in May. How '
                            'many clips did Natalia sell altogether in April '
                            'and May?'},
        outputs={'answer': '72'},
        adaptor_object=None),
 LMDemo(inputs={'question': 'Betty is saving money for a new wallet which '
                            'costs $100. Betty has only half of the money she '
                            'needs. Her parents decided to give her $15 for '
                            'that purpose, and her grandparents twice as much '
                            'as her parents. How much more money does Betty '
                            'need to buy the wallet?'},
        outputs={'answer': '5'},
        adaptor_object=None),
 LMDemo(inputs={'question': 'Julie is reading a 120-page book. Yesterday, she '
                            'was able to read 12 pages and today